# Deep Data Analysis (MC2)

This notebook provides deeper analysis of the data with the aim of spotting suspicious vessels using Data Mining tools and techniques to answer Q3 and Q4 of the challenge. The main goal is assessing similarity between **SouthSeafood Express Corp vessels** and other vessels, to identify potential links or patterns of suspicious behavior. More specifically: computing **similarity scores** and rank vessels according to risk.

In [49]:
import json
import pandas as pd
import numpy as np
import seaborn as sns
import altair as alt
from sklearn.metrics.pairwise import cosine_similarity
from tqdm import tqdm

### Load .json data and convert it to CSV

In [50]:
# Load .json data and read it as pandas DataFrame
folder_path = '../data/'

with open(folder_path + 'commodities.json') as f:
    commodities = pd.json_normalize(json.load(f))

with open(folder_path + 'vessels.json') as f:
    vessels = pd.json_normalize(json.load(f))

with open(folder_path + 'locations.json') as f:
    locations = pd.json_normalize(json.load(f))

with open(folder_path + 'documents.json') as f:
    documents = pd.json_normalize(json.load(f))

with open(folder_path + 'transponder_pings.json') as f:
    transponder_pings = pd.json_normalize(json.load(f))

with open(folder_path + 'harbor_reports.json') as f:
    harbor_reports = pd.json_normalize(json.load(f))

with open(folder_path + 'transactions.json') as f:
    transactions = pd.json_normalize(json.load(f))

Read timestamps

In [51]:
# Convert timestamps to datetime
transponder_pings['time'] = pd.to_datetime(transponder_pings['time'], errors='coerce')
transactions['date'] = pd.to_datetime(transactions['date'], errors='coerce')
harbor_reports['date'] = pd.to_datetime(harbor_reports['date'], errors='coerce')

# Convert dwell to numbers
transponder_pings['dwell'] = pd.to_numeric(transponder_pings['dwell'], errors='coerce')

## 1. Visualize Trajectories

In [52]:
# Sort values
transponder_pings_sorted = transponder_pings.sort_values(['target', 'time'])

trajectories = {}

for vessel, group in transponder_pings_sorted.groupby('target'):
    #traj = list(zip(group['time'], group['source'], group['dwell'])) # list of (time, location, dwell)
    traj = list(group['source'])
    trajectories[vessel] = traj

# Create a df for each vessel with its trajectory
vessel_groups = {v: g[['time','source','dwell']].reset_index(drop=True)
                 for v, g in transponder_pings_sorted.groupby('target')}

# Lists of all zones/ports
all_ports = list(set(transponder_pings_sorted['source']))
port_to_idx = {p: i for i, p in enumerate(all_ports)}

In [53]:
vessel_groups.keys()

dict_keys(['albacoreangler47d', 'albacoreassaulter482', 'amberjackassaulterd52', 'americaneelenthusiastcfa', 'anchovyassaulterb1c', 'aquaticangler5c3', 'aquaticpursuitf31', 'aquatransit6bc', 'arcticgraylingangler094', 'athenad34', 'atlanticbluemarlinmarauder0b4', 'atlanticcodcatcherca6', 'baitedbreath538', 'barracudabaiter8b3', 'barracudabandit836', 'bassbaiterb9f', 'bassbandit0d5', 'bigeyetunabanditb73', 'bigeyetunabuccaneera16', 'blackbullheadbandit801', 'blackdrumbanditc5b', 'bluecatfishcatcher468', 'bluefintunabandit177', 'bluefishbandit8ec', 'bluegillbandita5f', 'blueharbor2c1', 'bluemarlinbandit292', 'bonefishbaiter565', 'breambanditc85', 'brillbandit0a1', 'brinebuccaneer9fd', 'brooktroutbuccaneerc0b', 'brownbullheadbriganded2', 'browntroutbandite67', 'bulkcarriers6cd', 'burbotbandit7bf', 'cargocatalyst39a7', 'cargocatalystb02', 'cargocentric443e', 'cargocentric4d0', 'cargocircuit26cc', 'cargocircuit545', 'cargocosmoscde', 'cargocrestb7c', 'cargocynosure29d', 'carpcapturer993', '

In [54]:
vessel_groups.get('albacoreangler47d') # Example vessel trajectory data

,time,source,dwell
0,2035-02-01 00:00:00.000,South Paackland,0.000000
1,2035-02-01 05:00:00.000,Nav 3,1768.653072
2,2035-02-01 05:29:28.653,Cod Table,62410.977112
3,2035-02-01 22:49:39.630,Nav 2,6030.279121
4,2035-02-02 00:30:09.909,Cod Table,15499.004992
...,...,...,...
1230,2035-11-27 00:34:17.915,Nav 3,6697.277299
1231,2035-11-27 02:25:55.193,South Paackland,164281.048365
1232,2035-11-29 05:00:00.000,Nav 3,2858.998607
1233,2035-11-29 05:47:38.998,Tuna Shelf,19885.173842


Save trajectories in a separate file 'trajectories.json'

In [55]:
# # convert in string the datetime objects
# for v in vessel_groups:
#     vessel_groups[v]['time'] = vessel_groups[v]['time'].dt.strftime('%Y-%m-%dT%H:%M:%S.%f')
    
# # save vessel_groups to a json file 'trajectories.json' where each key is a vessel id and each value is a list of dictionaries with keys 'time', 'source', 'dwell'
# with open(folder_path + 'trajectories.json', 'w') as f:
#     json.dump({v: g[['time', 'source', 'dwell']].to_dict(orient='records') for v, g in vessel_groups.items()}, 
#               f,
#               default=str,
#               indent=2
#     )

### Load new 'trajectories.json'

In [56]:
with open(folder_path + 'trajectories.json', 'r') as f:
    trajectories = json.load(f)

Visualization Function

In [ ]:
def plot_vessel_dwell_timeline(
    trajectories: dict,
    vessel_id: str,
    location_to_y: dict,
    harbor_reports: pd.DataFrame,
    transactions: pd.DataFrame,
    documents: pd.DataFrame,
    vessel_name: str = None
):
    """
    Visualizes vessel trajectory with dwell times as bars,
    and overlays full-day port report bars and transaction markers.
    """

    # Check if the vessel exists
    if vessel_id not in trajectories:
        raise ValueError(f"Vessel '{vessel_id}' not found in trajectories.")

    # Prepare vessel trajectory DataFrame
    df = pd.DataFrame(trajectories[vessel_id]).copy()
    df['time'] = pd.to_datetime(df['time'], errors='coerce')
    df['dwell'] = pd.to_numeric(df['dwell'], errors='coerce')
    df = df.dropna(subset=['time', 'dwell', 'source'])
    df['end_time'] = df['time'] + pd.to_timedelta(df['dwell'], unit='s')
    df['dwell_hours'] = df['dwell'] / 3600
    df['y'] = df['source'].map(location_to_y)
    if vessel_name is None:
        vessel_name = vessel_id

    # Main bar chart for dwell times
    bars = alt.Chart(df).mark_bar().encode(
        x='time:T',
        x2='end_time:T',
        y=alt.Y('source:N', sort=list(location_to_y.keys()), title='Location'),
        color=alt.Color('source:N', legend=None),
        tooltip=['source', 'time', 'end_time', 'dwell_hours']
    )

    # --- Port reports ---
    vessel_reports = harbor_reports[harbor_reports['source'] == vessel_id].copy()
    vessel_reports['date'] = pd.to_datetime(vessel_reports['date'], errors='coerce')
    vessel_reports = vessel_reports.dropna(subset=['date'])
    vessel_reports = vessel_reports[vessel_reports['target'].isin(location_to_y.keys())]

    # --- Transactions ---
    vessel_transactions = transactions[
        transactions['suspected_vessels'].apply(lambda x: vessel_id in x if isinstance(x, list) else False)
    ].copy()
    vessel_transactions = vessel_transactions.dropna(subset=['date'])
    vessel_transactions = vessel_transactions[vessel_transactions['target'].isin(location_to_y.keys())]
    vessel_transactions['probable_cargo_transaction'] = True

    # Merge with documents
    vessel_transactions = vessel_transactions.merge(
        documents,
        left_on='source',
        right_on='id',
        how='left'
    )

    charts_to_combine = [bars]

    # Add report bars and markers if reports exist
    if not vessel_reports.empty:
        vessel_reports['day_start'] = vessel_reports['date'].dt.floor('D')
        vessel_reports['day_end'] = vessel_reports['date'].dt.floor('D') + pd.Timedelta(days=1)
        vessel_reports['day_mid'] = vessel_reports['day_start'] + pd.Timedelta(hours=12)

        report_bars = alt.Chart(vessel_reports).mark_bar(
            color='red',
            opacity=0.3
        ).encode(
            x='day_start:T',
            x2='day_end:T',
            y=alt.Y('target:N', sort=list(location_to_y.keys())),
            tooltip=['target', 'date', 'data_author']
        )

        markers_harbor = alt.Chart(vessel_reports).mark_circle(
            color='red',
            size=150,
            opacity=0.6
        ).encode(
            x='day_mid:T',
            y=alt.Y('target:N', sort=list(location_to_y.keys())),
            tooltip=['target', 'date', 'data_author']
        )

        charts_to_combine.extend([report_bars, markers_harbor])

    # Add transaction markers if transactions exist
    if not vessel_transactions.empty:
        vessel_transactions['day_mid'] = vessel_transactions['date'].dt.floor('D') + pd.Timedelta(hours=12)

        markers_transactions = alt.Chart(
            vessel_transactions[vessel_transactions['probable_cargo_transaction'] == True]
        ).mark_square(
            color='blue',
            size=100,
            opacity=0.6
        ).encode(
            x='day_mid:T',
            y=alt.Y('target:N', sort=list(location_to_y.keys())),
            tooltip=['target', 'date', 'qty_tons', 'commodity']
        )

        charts_to_combine.append(markers_transactions)

    # Combine all charts
    final_chart = alt.layer(*charts_to_combine).properties(
        width=950,
        height=300,
        title=f'Vessel Route Over Time: {vessel_name}'
    ).interactive()

    return final_chart

Let's apply the function to an example

In [58]:
vessel_id = 'snappersnatcher7be'
vessel_name = vessels.loc[vessels['id'] == vessel_id, 'name'].values[0]

chart = plot_vessel_dwell_timeline(
    trajectories=trajectories,
    vessel_id=vessel_id,
    location_to_y=port_to_idx,
    harbor_reports=harbor_reports,
    transactions=transactions,
    documents=documents,
    vessel_name=vessel_name
)
chart.display()


alt.LayerChart(...)

## 2. Visually associating cargo deliveries with vessels

In [59]:
# Merge transactions with documents
transactions_merged = transactions.merge(
    documents[['commodity', 'qty_tons']],
    left_on='source',
    right_on=documents['id'],
    how='left'
)

# Merge harbor reports with vessel details
harbor_reports_merged = harbor_reports.merge(
    vessels[['name', 'vessel_type', 'flag_country', 'company', 'tonnage', 'length_overall']],
    left_on='source',
    right_on=vessels['id'],
    how='left'
)

Create a plot in Altair showing seasonal trends of cargo deliveries and number of vessels in specific harbor over time.
The plot should have one x-axis representing time and two opposite y-axes: one for cargo deliveries (in tons) and another for number of vessels observed in that harbor.
The aim is to visualize potential correlations between cargo deliveries and vessel traffic in that harbor.
Use the harbor_reports and transactions dataframes to create the plot.

use different colors to distinguish between different types of fish commodities in the cargo deliveries.
use a stacked bar chart to represent the quantity of different fish commodities delivered each day.
use stacked bar chart to represent the tonnage of vessels observed in that harbor each day, with different colors for different vessels.
change the verse of the lower y-axis to represent the tonnage of vessels in harbor.
the upper y-axis should be over the x-axis, the lower y-axis should be under the x-axis. therefore the x-axis should be in the middle of the plot.
do not use the same color palette vor vessels, use another palette.

transactions_merged.columns is ['source', 'target', 'date', 'suspected_vessels', 'commodity',
       'qty_tons']

harbor_reports_merged.columns is 'source', 'target', 'date', 'data_author', 'name', 'vessel_type',
       'flag_country', 'company', 'tonnage', 'length_overall']

Prepare color palette

In [60]:
illegal_fish_species = {'piscessatisb87', 'piscesfoetidaae7', 'piscisosseusb6d'}

# Color palette for commodities
commodity_names = transactions_merged['commodity'].unique()
illegal_palette = sns.color_palette("Reds", len([name for name in commodity_names if name in illegal_fish_species]))
legal_palette = sns.color_palette("Blues", len([name for name in commodity_names if name not in illegal_fish_species]))

commodity_palette = {}
illegal_idx = 0
legal_idx = 0

for name in commodity_names:
    if name in illegal_fish_species:
        commodity_palette[name] = illegal_palette[illegal_idx]
        illegal_idx += 1
    else:
        commodity_palette[name] = legal_palette[legal_idx]
        legal_idx += 1

commodity_color_scale = alt.Scale(
    domain=list(commodity_palette.keys()),
    range=[f'#{int(r*255):02x}{int(g*255):02x}{int(b*255):02x}' 
           for r, g, b in commodity_palette.values()]
)

Cargo deliveries per day

In [ ]:
# Handle large datasets
alt.data_transformers.disable_max_rows()

# Transform suspected_vessels list to string for tooltip
transactions_merged['suspected_vessels_str'] = transactions_merged['suspected_vessels'].apply(
    lambda v: ', '.join(v) if isinstance(v, list) else str(v)
)

# Orden the shipments for convenience (date, harbor, commodity)
agg = transactions_merged.sort_values(['date', 'target', 'commodity'])

# cumulative sum
agg['cum_qty'] = agg.groupby(['date', 'target'])['qty_tons'].cumsum()
agg['cum_qty_prev'] = agg['cum_qty'] - agg['qty_tons']  # la base di ogni carico

# Dropdown for harbor selection
city_dropdown = alt.binding_select(
    options=agg['target'].dropna().unique().tolist(),
    name='Harbor: '
)
city_select = alt.selection_point(
    fields=['target'],
    bind=city_dropdown,
    value=agg['target'].dropna().iloc[0]
)

# Add selection for commodity
commodity_select = alt.selection_point(
    fields=['commodity'],
    bind='legend' 
)

bars = alt.Chart(agg).mark_bar(size=6).encode(
    x=alt.X('date:T', title='Date'),
    y=alt.Y('cum_qty_prev:Q', title='Quantity Imported (tons)'),
    y2='cum_qty:Q',
    color=alt.Color(
        'commodity:N',
        title='Fish Type',
        legend=alt.Legend(
            orient="right",
            columns=1,
            title="Fish Type",
            labelFontSize=12,
            titleFontSize=14
        ),
        scale=commodity_color_scale
    ),
    opacity=alt.condition(commodity_select, alt.value(1.0), alt.value(0.2)),
    tooltip=[
        alt.Tooltip('date:T', title='Date'),
        alt.Tooltip('commodity:N', title='Fish Type'),
        alt.Tooltip('qty_tons:Q', title='Shipment (tons)'),
        alt.Tooltip('target:N', title='Harbor'),
        alt.Tooltip('suspected_vessels_str:N', title='Suspected Vessels')
    ]
).add_params(
    city_select,
    commodity_select
).transform_filter(
    city_select
)

# Horizontal lines between cargo blocks
lines = alt.Chart(agg).mark_tick(
    orient='horizontal',
    color='black',
    size=10,
    thickness=0.7
).encode(
    x='date:T',
    y='cum_qty:Q',
    opacity=alt.condition(commodity_select, alt.value(1.0), alt.value(0.2)),
).add_params(
    city_select
).transform_filter(
    city_select
)

final_chart = alt.layer(bars, lines).properties(
    width=850,
    height=300,
    title='Fish Cargoes Imported per Day (individual shipments)'
).interactive()

final_chart.display()


alt.LayerChart(...)

Visualize vessels docked in Harbor

In [ ]:
# Handle large datasets
alt.data_transformers.disable_max_rows()

agg = harbor_reports_merged.sort_values(['date', 'target', 'vessel_type'])

# cumulative sum
agg['cum_tonnage'] = agg.groupby(['date', 'target'])['tonnage'].cumsum()
agg['cum_tonnage_prev'] = agg['cum_tonnage'] - agg['tonnage']

city_dropdown = alt.binding_select(
    options=agg['target'].dropna().unique().tolist(),
    name='Harbor: '
)
city_select = alt.selection_point(
    fields=['target'],
    bind=city_dropdown,
    value=agg['target'].dropna().iloc[0]
)

vessel_select = alt.selection_point(
    fields=['vessel_type'],
    bind='legend'
)

# Every bar is a ship segment stacked by tonnage
bars = alt.Chart(agg).mark_bar(size=6).encode(
    x=alt.X('date:T', title='Date'),
    y=alt.Y('cum_tonnage_prev:Q', title='Total Tonnage (tons)'),
    y2='cum_tonnage:Q',
    color=alt.Color(
        'vessel_type:N',
        title='Vessel Type',
        scale=alt.Scale(scheme='viridis'),
        legend=alt.Legend(
            orient="right",
            columns=1,
            title="Vessel Type",
            labelFontSize=12,
            titleFontSize=14
        )
    ),
    opacity=alt.condition(vessel_select, alt.value(1.0), alt.value(0.2)),
    tooltip=[
        alt.Tooltip('date:T', title='Date'),
        alt.Tooltip('name:N', title='Vessel Name'),
        alt.Tooltip('vessel_type:N', title='Vessel Type'),
        alt.Tooltip('flag_country:N', title='Flag Country'),
        alt.Tooltip('company:N', title='Company'),
        alt.Tooltip('tonnage:Q', title='Tonnage'),
        alt.Tooltip('length_overall:Q', title='Length Overall (m)')
    ]
).add_params(
    city_select,
    vessel_select
).transform_filter(
    city_select
)

# Horizontal lines between ship blocks
lines = alt.Chart(agg).mark_tick(
    orient='horizontal',
    color='black',
    size=10,
    thickness=0.7
).encode(
    x='date:T',
    y='cum_tonnage:Q',
    opacity=alt.condition(vessel_select, alt.value(1.0), alt.value(0.2))
).add_params(
    city_select,
    vessel_select
).transform_filter(
    city_select
)

final_chart = alt.layer(bars, lines).properties(
    width=850,
    height=400,
    title='Vessels Docked per Day (stacked by Tonnage)'
).interactive()

final_chart.display()


alt.LayerChart(...)

## 2. Compute similarity scores

Similarity functions

In [84]:
def jaccard_similarity(seq1, seq2):
    """Jaccard similarity based on zone/port visits"""
    set1, set2 = set(seq1), set(seq2)
    if not set1 or not set2:
        return 0.0
    return len(set1 & set2) / len(set1 | set2)

def dwell_vector(df):
    """Port dwell time vector"""
    vec = np.zeros(len(all_ports))
    for _, row in df.iterrows():
        port = row['source']
        if port is None:
            continue
        idx = port_to_idx[port]
        vec[idx] += row['dwell']
    return vec

Let's compute similarity w.r.t. *snappersnatcher7be*

In [85]:
ref_vessel = 'snappersnatcher7be'
ref_df = pd.DataFrame(trajectories[ref_vessel])
ref_df = ref_df[ref_df['source'].notna()]
ref_dwell_vec = dwell_vector(ref_df)
ref_ports = list(ref_df['source'])

similarities_jaccard = {}
similarities_dwell = {}

for vessel, df in tqdm(trajectories.items(), desc="Calculating similarities"):
    if vessel == ref_vessel:
        continue
    df = pd.DataFrame(df)
    df = df[df['source'].notna()]
    if df.empty:
        similarities_jaccard[vessel] = 0.0
        similarities_dwell[vessel] = 0.0
        continue  

    similarities_jaccard[vessel] = jaccard_similarity(list(df['source']), ref_ports) # Jaccard similarity

    vec = dwell_vector(df)
    similarities_dwell[vessel] = cosine_similarity([ref_dwell_vec], [vec])[0,0] # Cosine similarity on dwell vectors

# Combine similarities with weights
weight_jaccard = 0.6
weight_dwell = 0.4

final_similarity = {}
for vessel in similarities_jaccard:
    final_similarity[vessel] = weight_jaccard * similarities_jaccard[vessel] + weight_dwell * similarities_dwell[vessel]

# Sort by final similarity and print top 10
similar_vessels = sorted(final_similarity.items(), key=lambda x: x[1], reverse=True)
print("\nTop vessels similar to snappersnatcher7be:")
for vessel, sim in similar_vessels[:20]:
    print(f"{vessel}: {sim:.3f}")

Calculating similarities: 100%|██████████| 296/296 [00:13<00:00, 22.60it/s]


Top vessels similar to snappersnatcher7be:
swordfishsaboteur22f: 0.986
bigeyetunabuccaneera16: 0.964
fishfinderb9d: 0.963
browntroutbandite67: 0.962
salmonseeker630: 0.958
largemouthbasslooterf95: 0.956
cohosalmoncapturera7b: 0.953
prawnpredator5d7: 0.951
whitingwrangler842: 0.924
seabassbandit9ad: 0.899
whitemarlinwranglerbac: 0.877
crabcatcher1aa: 0.871
trawlertriumph31f: 0.869
aquaticpursuitf31: 0.867
codcatcher04c: 0.865
brillbandit0a1: 0.863
redfinpickerelraider744: 0.861
fishflingere29: 0.860
haddockhawkb7c: 0.857
spanishmackerelmaster037: 0.857
